In [1]:
import pandas as pd

In [16]:
path = '../exp/downloads/2024-02-14_11-37/11-37/'

output_path = path

csv_train_file_name = 'combined_train_data.csv'
csv_eval_file_name = 'combined_eval_data.csv'

agent_ids = ['follower0_0', 'follower1_0', 'follower2_0', 'follower3_0', 'follower4_0']
summable_train_keys = ['actor_entropy', 'actor_loss', 'actor_target_entropy', 'alpha_loss', 'alpha_value', 'batch_reward', 'critic_loss', 'duration', 'episode_reward']
static_train_keys = ['episode', 'step']
summable_eval_keys = ['episode_reward']
static_eval_keys = ['episode', 'step']

train_file_names = []
eval_file_names = []
for id in agent_ids:
    train_file_names.append(path + "train_" + id + ".csv")
    eval_file_names.append(path + "eval_" + id + ".csv")


In [38]:
paths = [
'../exp/downloads/2024-02-17_11-32/11-32/',
'../exp/downloads/2024-02-17_11-33/11-33/',
'../exp/downloads/2024-02-17_11-37/11-37/',
]

csv_train_file_name = 'train_fed_0.33_4.csv'
csv_eval_file_name = 'eval_fed_0.33_4.csv'

output_path = '../exp/downloads/'

summable_train_keys = ['actor_entropy', 'actor_loss', 'actor_target_entropy', 'alpha_loss', 'alpha_value', 'batch_reward', 'critic_loss', 'duration', 'episode_reward']
static_train_keys = ['episode', 'step']
summable_eval_keys = ['episode_reward']
static_eval_keys = ['episode', 'step']

train_file_names = []
eval_file_names = []
for path in paths:
    train_file_names.append(path + "combined_train_data.csv")
    eval_file_names.append(path + "combined_eval_data.csv")

In [39]:
# determine row number of smallest train file
row_counts = []
for file_index, file in enumerate(train_file_names):
    df = pd.read_csv(file)
    row_counts.append(len(df.index))
min_row_count = min(row_counts)

# prepare combined train file
train_data_dict = {}
for sum_key in summable_train_keys:
    train_data_dict[sum_key] = []
for static_key in static_train_keys:
    train_data_dict[static_key] = []

# aggregate train files
for file_index, file in enumerate(train_file_names):
    df = pd.read_csv(file)
    
    for index, row in df.iterrows():
        if index >= min_row_count:
            break
        
        for sum_key in summable_train_keys:
            if file_index == 0:
                train_data_dict[sum_key].append(row[sum_key])
            else:
                train_data_dict[sum_key][index] += row[sum_key]
        if file_index == 0:
            for static_key in static_train_keys:
                train_data_dict[static_key].append(row[static_key])

for sum_key in summable_train_keys:
    for row in range(len(train_data_dict[sum_key])):
        train_data_dict[sum_key][row] /= len(train_file_names)


# determine row number of smallest evaluation file
row_counts = []
for file_index, file in enumerate(eval_file_names):
    df = pd.read_csv(file)
    row_counts.append(len(df.index))
min_row_count = min(row_counts)

# prepare combined evaluation file
eval_data_dict = {}
for sum_key in summable_eval_keys:
    eval_data_dict[sum_key] = []
for static_key in static_eval_keys:
    eval_data_dict[static_key] = []

# aggregate evaluation files
for file_index, file in enumerate(eval_file_names):
    df = pd.read_csv(file)
    
    for index, row in df.iterrows():
        if index >= min_row_count:
            break

        for sum_key in summable_eval_keys:
            if file_index == 0:
                eval_data_dict[sum_key].append(row[sum_key])
            else:
                eval_data_dict[sum_key][index] += row[sum_key]
        if file_index == 0:
            for static_key in static_eval_keys:
                eval_data_dict[static_key].append(row[static_key])

for sum_key in summable_eval_keys:
    for row in range(len(eval_data_dict[sum_key])):
        eval_data_dict[sum_key][row] /= len(eval_file_names)


# output new combined files
train_df = pd.DataFrame.from_dict(train_data_dict)
eval_df = pd.DataFrame.from_dict(eval_data_dict)

train_df.to_csv(output_path + csv_train_file_name)
eval_df.to_csv(output_path + csv_eval_file_name)

